In [ ]:
from pyspark.sql import SparkSession

In [1]:
spark

In [2]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

In [3]:
data = spark.read.csv('hdfs://devenv/user/datasets/crime/data/', header=True, inferSchema=True)

In [5]:
data.show()

+---------+--------------------+--------------------+--------------------+-----+----+-----+
|lsoa_code|             borough|      major_category|      minor_category|value|year|month|
+---------+--------------------+--------------------+--------------------+-----+----+-----+
|E01004563|          Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
|E01003496|              Newham|     Criminal Damage|Criminal Damage T...|    0|2013|    9|
|E01000086|Barking and Dagenham|  Theft and Handling|  Other Theft Person|    1|2009|    5|
|E01001317|              Ealing|            Burglary|Burglary in a Dwe...|    0|2013|    9|
|E01000713|             Bromley|Violence Against ...|    Offensive Weapon|    0|2009|   12|
|E01000606|               Brent|  Theft and Handling|Motor Vehicle Int...|    0|2015|    9|
|E01003211|            Lewisham|  Theft and Handling|  Other Theft Person|    0|2009|   12|
|E01003902|Richmond upon Thames|     Criminal Damage|Criminal Damage T...|    0|

In [7]:
crime_2015 = data.filter('year>=2015').drop('lsoa_code')
crime_2015.show(10)

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|               Brent|  Theft and Handling|Motor Vehicle Int...|    0|2015|    9|
|       Tower Hamlets|            Burglary|Burglary in a Dwe...|    0|2016|    3|
|Richmond upon Thames|Violence Against ...|      Common Assault|    0|2015|    3|
|               Brent|  Theft and Handling|Theft/Taking of P...|    0|2015|    1|
|           Redbridge|            Burglary|Burglary in Other...|    0|2015|    2|
|             Bromley|     Criminal Damage|Criminal Damage T...|    0|2015|    9|
|          Hillingdon|Violence Against ...|        Wounding/GBH|    1|2015|    6|
|              Ealing|  Theft and Handling|  Other Theft Person|    0|2015|    8|
|           Greenwich|     Criminal Damage|Criminal Damage T...|    0|2015|    1|
|           Redb

In [10]:
convictions_by_brough = crime_2015.groupBy('borough').agg({'value':'sum'})
convictions_by_brough.show(10)

+--------------------+----------+
|             borough|sum(value)|
+--------------------+----------+
|             Croydon|      5669|
|          Wandsworth|      4791|
|              Bexley|      2513|
|             Lambeth|      6837|
|Barking and Dagenham|      3310|
|              Camden|      5790|
|           Greenwich|      4413|
|              Newham|      5807|
|       Tower Hamlets|      5855|
|              Barnet|      4960|
+--------------------+----------+
only showing top 10 rows



In [11]:
convictions_by_brough = convictions_by_brough.withColumnRenamed('sum(value)','num_of_convictions')
convictions_by_brough.show(10)

+--------------------+------------------+
|             borough|num_of_convictions|
+--------------------+------------------+
|             Croydon|              5669|
|          Wandsworth|              4791|
|              Bexley|              2513|
|             Lambeth|              6837|
|Barking and Dagenham|              3310|
|              Camden|              5790|
|           Greenwich|              4413|
|              Newham|              5807|
|       Tower Hamlets|              5855|
|              Barnet|              4960|
+--------------------+------------------+
only showing top 10 rows



In [14]:
total_convictions = convictions_by_brough.agg({'num_of_convictions':'sum'}).collect()[0][0]

In [19]:
from pyspark.sql.functions import *
convictions_by_borough_with_percentage = convictions_by_brough.withColumn("percentage_convictions",format_number(convictions_by_brough["num_of_convictions"]/total_convictions*100,2))
convictions_by_borough_with_percentage.show(10)

+--------------------+------------------+----------------------+
|             borough|num_of_convictions|percentage_convictions|
+--------------------+------------------+----------------------+
|             Croydon|              5669|                  3.88|
|          Wandsworth|              4791|                  3.28|
|              Bexley|              2513|                  1.72|
|             Lambeth|              6837|                  4.68|
|Barking and Dagenham|              3310|                  2.27|
|              Camden|              5790|                  3.97|
|           Greenwich|              4413|                  3.02|
|              Newham|              5807|                  3.98|
|       Tower Hamlets|              5855|                  4.01|
|              Barnet|              4960|                  3.40|
+--------------------+------------------+----------------------+
only showing top 10 rows



In [ ]:
# convictions_by_borough_with_percentage.persist()

# # show result of 100 records to console
# convictions_by_borough_with_percentage.show(100)

# write result to MySQL Table convictions_by_borough_with_percentage
convictions_by_borough_with_percentage.write \
                          .option("driver", "com.mysql.jdbc.Driver") \
                          .jdbc("jdbc:mysql://localhost:3306", "crime.convictions_by_borough_with_percentage",
                                properties={"user": "spark", "password": "1qaz@wsX"})